In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from functools import partial
import pandas as pd
from IPython.display import display
import ipywidgets as widgets

## Control of oscillations (OOP version)

In [2]:
#calc_others(sol, consts)
class oscillation:
    def __init__(self, model, params, consts, calc_all):

        #Model and parameters
        self.__model = model
        self.__params = params
        self.__consts = consts
        self.__calc_all = calc_all

        #Essential information
        self.__species = ['A2', 'S_total', 'A', 'O']
        self.__info = None

        #Experimental data
        self.__exp_data = None

    @property
    def info(self):
        print(f'The model includes {len(self.__params)} parameters and {len(self.__consts)} constants. The species are {self.__species}.')
        print(f'Additional information: {self.__info}')

    def add_info(self, info):
        self.__info = info
        
    def simulate(self, init_cond, t=10):
        params_pass = self.__params + self.__consts
        model_partial = partial(self.__model, params = params_pass)
        t_span = (0, t)
        t_eval = np.linspace(0, t, 250)
        sol = solve_ivp(model_partial, t_span = t_span, y0 = init_cond, t_eval = t_eval, rtol=1e-6, atol=1e-8)
        return sol
    
    def plot(self, init_cond, t=10):
        sol = self.simulate(init_cond, t)
        c = self.__calc_all(sol, self.__consts)
        fig, ax = plt.subplots(2, 2, figsize = (10,6))
        i = 0
        for ax, y in zip(ax.flatten(), c):
            ax.plot(sol.t, y, label = self.__species[i])
            ax.legend()
            i += 1
        return fig, ax
    
    def interactive_plot(self, init_cond, t=10, range = 5, step=0.05):
        
        if len(self.__params) == 4:
            print('4 parameters')
            def plot_temp(alpha, beta, theta, phi):
                params_old = self.__params
                params = [alpha, beta, theta, phi]
                self.__params = params
                self.plot(init_cond, t)
                self.__params = params_old
            
            alpha, beta, theta, phi = self.__params
            alpha_slider = widgets.FloatSlider(value=alpha, min=max(0, alpha-range), max=alpha+range, step=step, description='alpha')
            beta_slider = widgets.FloatSlider(value=beta, min=max(0, beta-range), max=beta+range, step=step, description='beta')
            theta_slider = widgets.FloatSlider(value=theta, min=max(0, theta-range), max=theta+range, step=step, description='theta')
            phi_slider = widgets.FloatSlider(value=phi, min=max(0, phi-range), max=phi+range, step=step, description='phi')

            interactive_widget = widgets.interactive(plot_temp, alpha=alpha_slider, beta=beta_slider, theta=theta_slider, phi=phi_slider)
            display(interactive_widget)

        elif len(self.__params) == 5:
            print('5 parameters')
            pass

    def add_exp_data(self, data):
        print(f'The species are {self.__species}. Please check if the data is in the same order.')
        self.__exp_data = data

    def fit(self):
        pass

In [3]:
#Params includes alpha, beta, theta, phi, lam, m in the final model

def model1(t, vars, params):
    alpha, beta, theta, phi, lam, m = params
    C_A2, C_S = vars
    dC_A2dt = 1 - alpha * C_A2 * C_S**m - theta * C_A2
    dC_Sdt = alpha/lam * C_A2 * C_S**m - beta * C_S**(m + 1) + theta/lam * C_A2 - phi * C_S
    return (dC_A2dt, dC_Sdt)

def model1_Hill(t, vars, params):
    pass

def calculate_all_conc(sol, consts):
    lam, m = consts
    C_A2 = sol.y[0]
    C_S = sol.y[1]
    C_M = C_S ** m
    C_A = 2 * (1 - C_A2) - lam * (C_S + C_M)
    C_O = 1 / (2 * (1 - C_A2) - lam * (C_S + C_M)) ** 2
    return (C_A2, C_S + C_M, C_A, C_O)

In [4]:
params = [10, 10, 10, 10]
consts = [0.17, 4]
init_cond = [0.1, 0]

model_test = oscillation(model1, params, consts, calculate_all_conc)

In [5]:
model_test.add_info('This is a test model. The original Fernando\'s model')
model_test.info

The model includes 4 parameters and 2 constants. The species are ['A2', 'S_total', 'A', 'O'].
Additional information: This is a test model. The original Fernando's model


In [6]:
model_test.interactive_plot(init_cond, range=10, t=6)

4 parameters


interactive(children=(FloatSlider(value=10.0, description='alpha', max=20.0, step=0.05), FloatSlider(value=10.…